In [1]:
import numpy as np
import cvxpy
import lab

In [126]:
## This is a basic CVXPY based implementation on a toy dataset for the paper 
## "Neural Networks are Convex Regularizers: Exact Polynomial-time Convex Optimization Formulations for Two-layer Networks"
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt


def relu(x):
    return np.maximum(0,x)
def drelu(x):
    return x>=0
n=10
d=3
X=np.random.randn(n,d-1)
X=np.append(X,np.ones((n,1)),axis=1)

y=((np.linalg.norm(X[:,0:d-1],axis=1)>1)-0.5)*2
beta=1e-4


dmat=np.empty((n,0))

## Finite approximation of all possible sign patterns
for i in range(int(1e2)):
    u=np.random.randn(d,1)
    dmat=np.append(dmat,drelu(np.dot(X,u)),axis=1)

dmat=(np.unique(dmat,axis=1))


# Optimal CVX
m1=dmat.shape[1]
Uopt1=cp.Variable((d,m1))
Uopt2=cp.Variable((d,m1))

## Below we use hinge loss as a performance metric for binary classification
yopt1=cp.Parameter((n,1))
yopt2=cp.Parameter((n,1))
yopt1=cp.sum(cp.multiply(dmat,(X*Uopt1)),axis=1)
yopt2=cp.sum(cp.multiply(dmat,(X*Uopt2)),axis=1)
cost=cp.sum(cp.norm2((y - yopt1 + yopt2)) ** 2)/n+beta*(cp.mixed_norm(Uopt1.T,2,1)+cp.mixed_norm(Uopt2.T,2,1))
constraints=[]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt1))>=0]
constraints+=[cp.multiply((2*dmat-np.ones((n,m1))),(X*Uopt2))>=0]
prob=cp.Problem(cp.Minimize(cost),constraints)
prob.solve()
cvx_opt=prob.value
print("Convex program objective value (eq (8)): ",cvx_opt)

Convex program objective value (eq (8)):  0.001079083847223075


C:\Users\Strai\miniconda3\envs\ECE-285-Project\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 37 times so far.

  warnings.warn(msg, UserWarning)
C:\Users\Strai\miniconda3\envs\ECE-285-Project\lib\site-packages\cvxpy\expressions\expression.py:612: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 38 times so far.

  warnin

In [146]:
import torch
torch_x = torch.from_numpy(X).float()
torch_y = torch.from_numpy(y).float()
torch_dmat = torch.from_numpy(dmat).float()
torch_v = torch.nn.Parameter(data=torch.from_numpy(Uopt1.value).float(), requires_grad=True)
torch_w = torch.nn.Parameter(data=torch.from_numpy(Uopt2.value).float(), requires_grad=True)
print(torch_v.shape)
step_size = 1e-3
previous_loss = 1000
for _ in range(100):
    V = torch.sum(torch.multiply(torch_dmat, (torch_x @ torch_v)), axis=1)
    W = torch.sum(torch.multiply(torch_dmat, (torch_x @ torch_w)), axis=1)
    yhat = V - W

    temp = torch.sum(torch.linalg.norm(torch_y - yhat) ** 2) / n
    temp = temp + beta * (torch.linalg.norm(torch_v.T) + torch.linalg.norm(torch_w.T))
    print(temp.item())
    temp.backward()
    torch_w.data = torch_w.data - step_size * torch_w.grad.data
    torch_v.data = torch_v.data - step_size * torch_v.grad.data

    torch_w.grad.zero_()
    torch_v.grad.zero_()


    previous_loss = temp.item()


V1 = torch.sum(torch.multiply(torch_dmat, (torch_x @ torch_v)), axis=1)
W1 = torch.sum(torch.multiply(torch_dmat, (torch_x @ torch_w)), axis=1)
V1 - W1

torch.multiply((2*torch_dmat-torch.ones((n,m1))),(torch_x @ torch_v))>=0


torch.Size([3, 38])
0.0004919954226352274
0.0004919904749840498
0.0004919855273328722
0.0004919806960970163
0.0004919758648611605
0.0004919710336253047
0.0004919664934277534
0.0004919617786072195
0.0004919571802020073
0.0004919525817967951
0.0004919480998069048
0.0004919435014016926
0.0004919390194118023
0.0004919346538372338
0.0004919302300550044
0.0004919259226880968
0.0004919214989058673
0.0004919173661619425
0.0004919131170026958
0.0004919088096357882
0.0004919045604765415
0.0004919004859402776
0.0004918964114040136
0.0004918922204524279
0.0004918882041238248
0.0004918841877952218
0.0004918801714666188
0.0004918762715533376
0.0004918724880553782
0.0004918684717267752
0.0004918646300211549
0.0004918609047308564
0.000491857121232897
0.0004918533959425986
0.0004918496124446392
0.0004918459453620017
0.0004918422782793641
0.0004918385529890656
0.0004918350023217499
0.0004918315098620951
0.0004918279009871185
0.0004918244085274637
0.0004918209160678089
0.000491817481815815
0.000491814047

tensor([[ True,  True, False,  True,  True, False, False,  True,  True,  True,
          True, False, False, False,  True,  True,  True,  True, False, False,
          True, False, False,  True,  True, False, False, False,  True, False,
          True,  True, False, False, False, False, False, False],
        [ True, False,  True,  True,  True, False, False, False,  True, False,
          True,  True, False,  True,  True, False, False, False,  True,  True,
         False,  True,  True, False, False,  True, False,  True,  True, False,
          True,  True,  True, False, False,  True,  True, False],
        [ True, False, False, False, False, False, False, False, False, False,
         False, False, False, False,  True,  True,  True, False,  True,  True,
          True,  True,  True,  True,  True, False, False, False, False, False,
          True,  True,  True, False, False, False, False, False],
        [ True, False,  True,  True,  True,  True,  True,  True, False, False,
         Fal

In [3]:
(X @ Uopt1.value).shape, dmat.shape

((10, 37), (10, 37))

In [132]:
u = np.random.randn(*Uopt1.value.shape)
v = np.random.randn(*Uopt2.value.shape)
U = np.sum(np.multiply(dmat, (X @ Uopt1.value)), axis=1)
V = np.sum(np.multiply(dmat, (X @ Uopt2.value)), axis=1)
U - V, y, V1 - W1

(array([ 1.00014247,  1.00020982, -0.99925715,  0.99975448, -0.99929034,
        -0.99857581,  0.99882351,  0.99861577, -1.00085644, -0.99907474]),
 array([ 1.,  1., -1.,  1., -1., -1.,  1.,  1., -1., -1.]),
 tensor([ 1.0000,  1.0002, -0.9995,  1.0000, -0.9996, -0.9990,  0.9991,  0.9990,
         -1.0008, -0.9992], grad_fn=<SubBackward0>))

In [5]:
def h(X, W1, W2):
    Z = X @ W1.T
    return np.max(Z, 0) @ W2.T

In [6]:
def relu_solution_mapping(weights, remove_sparse: bool = False):
    assert len(weights.shape) == 4

    weight_norms = (lab.sum(weights ** 2, axis=-1, keepdims=True)) ** (1 / 4)
    normalized_weights = lab.safe_divide(weights, weight_norms)

    num_classes = weights.shape[1]
    first_layer = None
    second_layer = []
    for c in range(num_classes):
        pre_zeros = [
            lab.zeros_like(weight_norms[0, c]) for i in range(2 * c)
        ]  # positive neurons
        post_zeros = [
            lab.zeros_like(weight_norms[0, c])
            for i in range(2 * (num_classes - c - 1))
        ]

        if first_layer is None:
            pre_weights = []
        else:
            pre_weights = [first_layer]

        first_layer = lab.concatenate(
            pre_weights
            + [
                normalized_weights[0][c],
                normalized_weights[1][c],
            ],
            axis=0,
        )

        w2 = lab.concatenate(
            pre_zeros
            + [
                weight_norms[0][c],
                -weight_norms[1][c],
            ]
            + post_zeros,
            axis=0,
        ).T
        second_layer.append(w2)

    second_layer = lab.concatenate(second_layer, axis=0)

    if remove_sparse:
        sparse_indices = lab.sum(first_layer, axis=1) != 0

        first_layer = first_layer[sparse_indices]
        second_layer = second_layer[:, sparse_indices]

    return first_layer, second_layer

In [133]:
u = torch_w.detach().numpy()
v = torch_v.detach().numpy()
print(np.sum(u - Uopt1.value))
weights = np.asarray([u.T[np.newaxis, ...], v.T[np.newaxis, ...]])
weights.shape

-0.8943468481849357


(2, 1, 38, 3)

In [134]:
first_layer, second_layer = relu_solution_mapping(weights)
first_layer.shape, second_layer.shape

((76, 3), (1, 76))

In [142]:
y_hat = np.maximum(X @ first_layer.T, 0) @ second_layer.T
y_hat.flatten(), (W1 - V1).detach().numpy().flatten()

(array([-1.00008394, -1.00024645,  0.99923569, -0.9998878 ,  0.99932438,
         0.99858516, -0.99885033, -0.99857286,  1.00086481,  0.99896518]),
 array([-1.0000315 , -1.0002092 ,  0.9994532 , -0.99995315,  0.9996407 ,
         0.9989744 , -0.99906576, -0.9990246 ,  1.0008032 ,  0.9991805 ],
       dtype=float32))

In [11]:
np.linalg.norm(y_hat - y) / 2

9.431263687803744

In [12]:
np.linalg.norm(first_layer, axis=1)

array([5.35196622e-05, 8.84419883e-05, 5.37521787e-05, 2.07437063e-04,
       1.00073661e-04, 1.05839852e-04, 1.06817992e-04, 5.99995749e-05,
       3.66983459e-04, 3.21400563e-04, 3.75653638e-01, 1.31702164e+00,
       8.08135707e-05, 9.29051668e-05, 7.26736617e-05, 6.52529665e-05,
       5.09129867e-05, 4.75941400e-05, 5.87642561e-05, 8.50540817e-01,
       7.66011602e-01, 3.08373013e-04, 1.07950341e-04, 1.16344173e+00,
       7.45397194e-05, 2.37792862e-04, 7.94876247e-05, 7.26755656e-05,
       1.33518825e-03, 1.33518682e-03, 7.49985581e-05, 3.85312993e-05,
       6.26299270e-05, 5.99172624e-05, 3.81812373e-05, 6.39062917e-05,
       4.81448639e-05, 5.35196622e-05, 5.16749107e-05, 5.33242822e-05,
       4.78307743e-05, 5.17301424e-05, 4.94403369e-05, 4.45873737e-05,
       5.41328079e-05, 3.95143690e-05, 4.12333694e-05, 3.78057851e-05,
       3.93295325e-05, 5.03331748e-05, 4.59129701e-05, 1.60853516e-04,
       9.47347835e-05, 1.27411479e-04, 2.88111964e-04, 5.21693194e-05,
      

In [13]:
np.max(1 - np.multiply(y.flatten(), y_hat.flatten())

SyntaxError: unexpected EOF while parsing (2869790986.py, line 1)